In [3]:
import os
from pathlib import Path
import numpy as np

summary_path = Path("../data_raw/chb_subset/chb01/chb01-summary.txt")

seizure_intervals = []

with open(summary_path, "r") as f:
    for line in f:
        line = line.strip()
        if "Seizure" in line and "Start Time" in line:
            start = int(line.split(":")[1].strip())
        if "Seizure" in line and "End Time" in line:
            end = int(line.split(":")[1].strip())
            seizure_intervals.append((start, end))

print("Seizure intervals:", seizure_intervals)

ValueError: invalid literal for int() with base 10: '2996 seconds'

In [4]:
import os
from pathlib import Path
import numpy as np

summary_path = Path("../data_raw/chb_subset/chb01/chb01-summary.txt")

seizure_intervals = []

def extract_number(text):
    # Extract ONLY digits from the line
    digits = "".join(ch for ch in text if ch.isdigit())
    return int(digits)

with open(summary_path, "r") as f:
    start, end = None, None
    
    for line in f:
        line = line.strip()

        if "Seizure" in line and "Start Time" in line:
            start = extract_number(line)

        if "Seizure" in line and "End Time" in line:
            end = extract_number(line)

            # only append if we have both values
            if start is not None and end is not None:
                seizure_intervals.append((start, end))
                start, end = None, None

print("Seizure intervals:", seizure_intervals)

Seizure intervals: [(2996, 3036), (1467, 1494), (1732, 1772), (1015, 1066), (1720, 1810), (327, 420), (1862, 1963)]


In [5]:
import numpy as np

# Already computed earlier:
# windows.shape = (720, 23, 1280)
# Each window = 5 seconds (since 1280 samples @ 256Hz)

window_size_sec = 5
n_windows = windows.shape[0]

labels = np.zeros(n_windows, dtype=int)

for i in range(n_windows):
    start_t = i * window_size_sec
    end_t   = (i+1) * window_size_sec

    for (sz_start, sz_end) in seizure_intervals:
        if end_t >= sz_start and start_t <= sz_end:
            labels[i] = 1
            break

print("Total seizure windows:", labels.sum())
print("labels shape:", labels.shape)

NameError: name 'windows' is not defined

In [6]:
# --- LOAD EDF AGAIN ---
import mne
from pathlib import Path
import numpy as np

edf_path = Path("../data_raw/chb_subset/chb01/chb01_01.edf")
raw = mne.io.read_raw_edf(edf_path, preload=True)

# --- FILTER ---
raw.filter(1, 40)

# --- CREATE WINDOWS (same as earlier) ---
data = raw.get_data()
sfreq = raw.info['sfreq']
window_size = int(5 * sfreq)   # 5-second windows
n_samples = data.shape[1]
n_channels = data.shape[0]

n_windows = n_samples // window_size
windows = np.zeros((n_windows, n_channels, window_size))

for i in range(n_windows):
    start = i * window_size
    end = start + window_size
    windows[i] = data[:, start:end]

print("windows.shape =", windows.shape)

Extracting EDF parameters from /Users/deepakreddy/epilepsy_project/data_raw/chb_subset/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 s)



/var/folders/gj/g3mlkc4s68n3jy_tv4cn069w0000gn/T/ipykernel_67470/2315999467.py:7: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True)


windows.shape = (720, 23, 1280)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

# X is your feature matrix (720, 69)
# labels is your y

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
prob = clf.predict_proba(X_test)[:,1]

print("Accuracy:", accuracy_score(y_test, pred))
print("ROC AUC:", roc_auc_score(y_test, prob))

NameError: name 'X' is not defined

In [8]:
print(X.shape)      # should be (720, 69)
print(labels.shape) # should be (720,)

NameError: name 'X' is not defined

In [9]:
print("Feature matrix shape:", X.shape)
np.save(processed_dir / "chb01_01_features.npy", X)

NameError: name 'X' is not defined

In [10]:
import numpy as np
from pathlib import Path

data_dir = Path("../data_raw/chb_subset/chb01")
processed_dir = Path("../data_processed")
processed_dir.mkdir(exist_ok=True)

# seizure intervals we extracted
seizure_intervals = [(2996, 3036), (1467, 1494), (1732, 1772),
                     (1015, 1066), (1720, 1810), (327, 420), (1862, 1963)]

window_size_sec = 5

edf_files = sorted([f for f in data_dir.glob("*.edf")])
print("Found EDF files:", [f.name for f in edf_files])

for f in edf_files:
    base = f.stem  # e.g., "chb01_01"
    X_path = processed_dir / f"{base}_features.npy"
    
    if not X_path.exists():
        print("❌ Missing features for", base)
        continue

    X = np.load(X_path)
    n_windows = X.shape[0]

    window_centers_sec = np.arange(n_windows) * window_size_sec + window_size_sec / 2

    labels = np.zeros(n_windows, dtype=int)
    for start, end in seizure_intervals:
        labels[(window_centers_sec >= start) & (window_centers_sec <= end)] = 1

    # SAVE LABELS
    label_path = processed_dir / f"{base}_labels.npy"
    np.save(label_path, labels)

    print(f"Saved labels → {label_path}")

Found EDF files: ['chb01_01.edf', 'chb01_02.edf', 'chb01_03.edf', 'chb01_04.edf', 'chb01_05.edf', 'chb01_06.edf', 'chb01_07.edf', 'chb01_08.edf', 'chb01_09.edf', 'chb01_10.edf', 'chb01_11.edf', 'chb01_12.edf', 'chb01_13.edf', 'chb01_14.edf', 'chb01_15.edf', 'chb01_16.edf', 'chb01_17.edf', 'chb01_18.edf', 'chb01_19.edf', 'chb01_20.edf', 'chb01_21.edf', 'chb01_22.edf', 'chb01_23.edf', 'chb01_24.edf', 'chb01_25.edf', 'chb01_26.edf', 'chb01_27.edf', 'chb01_29.edf', 'chb01_30.edf', 'chb01_31.edf', 'chb01_32.edf', 'chb01_33.edf', 'chb01_34.edf', 'chb01_36.edf', 'chb01_37.edf', 'chb01_38.edf', 'chb01_39.edf', 'chb01_40.edf', 'chb01_41.edf', 'chb01_42.edf', 'chb01_43.edf', 'chb01_46.edf']
Saved labels → ../data_processed/chb01_01_labels.npy
Saved labels → ../data_processed/chb01_02_labels.npy
Saved labels → ../data_processed/chb01_03_labels.npy
Saved labels → ../data_processed/chb01_04_labels.npy
Saved labels → ../data_processed/chb01_05_labels.npy
Saved labels → ../data_processed/chb01_06_lab

In [11]:
from pathlib import Path
import numpy as np

processed_dir = Path("../data_processed")

X_list = []
y_list = []

# ONLY 8 recordings
for i in range(1, 9):          # 1..8
    rec_id = f"chb01_{i:02d}"  # chb01_01, chb01_02, ...
    X_file = processed_dir / f"{rec_id}_features.npy"
    y_file = processed_dir / f"{rec_id}_labels.npy"

    X_list.append(np.load(X_file))
    y_list.append(np.load(y_file))

X = np.vstack(X_list)
labels = np.concatenate(y_list)

print("X shape:", X.shape)         # expected (n_windows, 69)
print("labels shape:", labels.shape)  # expected (n_windows,)

X shape: (5760, 69)
labels shape: (5760,)


In [12]:
np.save(processed_dir / "X.npy", X)
np.save(processed_dir / "labels.npy", labels)
print("Saved X and labels")

Saved X and labels


In [13]:
X = np.load(processed_dir / "X.npy")
labels = np.load(processed_dir / "labels.npy")
print(X.shape, labels.shape)

(5760, 69) (5760,)
